In [2]:
import pandas as pd
import numpy as np

from time import time
from string import digits
import random
import threading

from geopy import Point
from geopy import Nominatim
from postal.parser import parse_address
from postal.expand import expand_address
import googlemaps
import warnings
# from googlemaps_api_key import key 

# key = key.replace('†','')
key = ''
gmaps = googlemaps.Client(key=key)
# https://console.cloud.google.com/apis/credentials?project=la-api-key


 API queries_quota: 60 



In [3]:
cd 

/Users/karatechop


In [4]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE


In [5]:
ls

ALL-BPD-AB953-DUSK.csv      ALL-OAKLAND_PD-AB953.csv
ALL-BPD-AB953.csv           ALL-SDPD-AB953-DUSK.csv
ALL-MARIN-AB953-DUSK.csv    ALL-SDPD-AB953.csv
ALL-MARIN-AB953.csv         ALL-SFPD-AB953-DUSK.csv
ALL-OAKLAND-AB953-DUSK.csv  ALL-SFPD-AB953.csv


## Function Setup

In [6]:
# def find_lat_lon_GMV3_debuggARR(city,address):
#     bounds = gmaps.geocode(city)[0]['geometry']['bounds']
    
#     if address != None: 
#         try:
#             return gmaps.geocode(address,bounds=bounds)

#         except:
#             print(f'Google maps failed for {address}')
          
#             pass

In [20]:
def find_lat_lon_GMV3(city,df,location_col='location'):
    df['neighborhood'],df['zip'],df['coords'] = None,None,None
    gmv3_total_start = time()
    counter = 0
    
    global failed_addresses, failed_counter
    failed_addresses = []
    failed_counter = 0
    
    bounds = gmaps.geocode(city)[0]['geometry']['bounds']
    
    for address in df[location_col].unique(): 
        if address != None and 'freeway' not in address: 
            try:
                result = gmaps.geocode(address,bounds=bounds)
                
                df.loc[df[location_col]==address,'gmap address'] = result[0]['formatted_address']
                df.loc[df[location_col]==address,'neighborhood'] = result[0]['address_components'][1]['long_name']
                df.loc[df[location_col]==address,'zip'] = result[0]['address_components'][-1]['long_name']
                df.loc[df[location_col]==address,'coords'] = str(result[0]['geometry']['location']['lat']) + ', ' +\
                                                             str(result[0]['geometry']['location']['lng'])
            
            except:
#                 print(f'Google maps failed for {address}')
                
                failed_addresses.append(address)
                failed_counter+=1
                
                pass
        
        
        counter+=1
        if counter%500==0:
            time_passed = round((time()-gmv3_total_start)/60,2)
            print(f'Counter is at {counter} for {city}. {time_passed} minutes have passed.')

## WAVE 1

In [17]:
SFPD = pd.read_csv('ALL-SFPD-AB953-DUSK.csv')
SFPD = SFPD[SFPD['itp']==1]

SFPD = SFPD.sort_values(by='location')
SFPD['location'] = SFPD['location'].str.lower()
SFPD.loc[SFPD['location'].str.contains('101'),'location'] = '101 freeway'

SFPD.shape

(24107, 14)

In [18]:
SFPD1 = SFPD.iloc[:int(SFPD.shape[0]/3),:]
SFPD2 = SFPD.iloc[int(SFPD.shape[0]/3):int(SFPD.shape[0]/3)*2,:]
SFPD3 = SFPD.iloc[int(SFPD.shape[0]/3)*2:int(SFPD.shape[0]/3)*3,:]

In [25]:
len(SFPD['location'].unique())

12151

In [ ]:
# find_lat_lon_GMV3('San Francisco',SFPD,'location')
SFPD.sample(5)

Google maps failed for N/A..
Google maps failed for PENNSYLVANIA AVE & MARIPOSA ST
Google maps failed for Trillium CNG Lot


In [21]:
warnings.filterwarnings("ignore") 

if __name__ =="__main__":
    t1 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD1))
    t2 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD2))
    t3 = threading.Thread(target=find_lat_lon_GMV3,args=('SAN FRANCISCO',SFPD3))
    
    t1.start()
    t2.start()
    t3.start()
 
    t1.join()
    t2.join()
    t3.join()
 
    # both threads completely executed
    print("\n\n\nBoth threads are done!\n\n")

Counter is at 500 for SAN FRANCISCO. 1.01 minutes have passed.
Counter is at 500 for SAN FRANCISCO. 1.01 minutes have passed.
Counter is at 500 for SAN FRANCISCO. 1.01 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 2.1 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 2.14 minutes have passed.
Counter is at 1000 for SAN FRANCISCO. 2.16 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 3.15 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 3.24 minutes have passed.
Counter is at 1500 for SAN FRANCISCO. 3.24 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 4.23 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 4.26 minutes have passed.
Counter is at 2000 for SAN FRANCISCO. 4.3 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 5.27 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 5.34 minutes have passed.
Counter is at 2500 for SAN FRANCISCO. 5.38 minutes have passed.
Counter is at 3000 for SAN FRANCISCO. 6.32 mi

In [31]:
SFPD_converted = pd.concat([SFPD1,SFPD2,SFPD3])
SFPD_converted.isna().sum()

date                   0
time                   0
Experience             0
race                   0
type                  29
location               0
sunsettime             0
dusktime               0
mins_sunset            0
mins_dusk              0
mins_stop_time         0
mins_before_dusk       0
itp                    0
dark                   0
neighborhood        1952
zip                 1952
coords              1952
gmap address        1946
dtype: int64

In [ ]:
def formatter(address):
    address = address.replace('/',' & ')
    address = address.replace(' / ',' & ')
    address = address.replace('/ ',' & ')
    address = address.replace(' @ ',' & ')
    address = address.replace(' and ',' & ')
    
    if 'terminal' in address:
        address = 'SFO'
    if 'upper' in address or 'lower' in address or 'sfia' in address:
        address = 'SFO'
    
    if address.split()[-1].isalpha:
        address += 'th'
        
    return address

['columbus/washington',
 'congdon/silver',
 '00 block of grove street',
 '00 block of  cervantes',
 'courtyard 3',
 '00 block of 4th street',
 '000 blk 7th st.',
 'cple  /  wallwr',
 '000 blk of 7th st.',
 'crescent/presita',
 'curtis/morse',
 '02nd st / south park',
 'oakdale and griffith',
 'oakdale/baldwin',
 'octavia @ oak',
 'octavia and grove',
 'octavia and laguna',
 'octavia and bay',
 'on 02/12/2019 i conducted a traffic stop because the drivers registration tabs were not present on the vehicle.',
 'driving without headlights during',
 '07th st / cleveland st',
 'departure level itb door #4',
 'departure level itb door #5',
 'departure level terminal 3 center island',
 'dimond / kern',
 'disobeying a sign',
 '09th st / division st',
 'division and isis',
 '100 4th street',
 '100 5th street',
 'dolores and guerrero',
 '100 block of brookdale',
 'dolores/25th st',
 'pennsylvania ave / 25th st',
 'domestic lower level t2',
 '100 block jefferson',
 'driver drove through a red ligh

In [ ]:
for i in SFPD_converted[SFPD_converted['neighborhood'] is None]['location']: